<a id='0'></a>
# Part 1. Problem Definition

This problem is to design an automated trading solution for single stock trading. We model the stock trading process as a Markov Decision Process (MDP). We then formulate our trading goal as a maximization problem.

The algorithm is trained using Deep Reinforcement Learning (DRL) algorithms and the components of the reinforcement learning environment are:


* Action: 
* Reward function: 
* State: 
* Environment: 

<a id='1'></a>
# Part 2. Getting Started- Load Python Packages

<a id='1.1'></a>
## 2.1. Install all the packages

In [1]:
!pip install -r requirements.txt --user

  Cloning git://github.com/dvdpedros/pyfolio (to revision fix-extreme-drawdown) to c:\users\adega\appdata\local\temp\pip-req-build-p8xht4c5
  Running command git checkout -b fix-extreme-drawdown --track origin/fix-extreme-drawdown
  Branch 'fix-extreme-drawdown' set up to track remote branch 'fix-extreme-drawdown' from 'origin'.
  Switched to a new branch 'fix-extreme-drawdown'



<a id='1.2'></a>
## 2.2. Import Packages

In [187]:
%load_ext autoreload
%autoreload 
import pandas as pd
from config import config
from dataset.download_dataset.cryptodownloader_binance import CryptoDownloader_binance
from preprocessing.preprocessors import FeatureEngineer
from preprocessing.data import data_split
from env.env_custom import CustomTradingEnv
from env.env_stocktrading import StockTradingEnv
from model.models import DRLAgent
from trade.backtest import BackTest
import warnings
warnings.filterwarnings("ignore")

# from trade.backtest import backtest_stats, backtest_plot, get_daily_return, get_baseline,convert_daily_return_to_pyfolio_ts


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<a id='1.3'></a>
## 2.3 Create Folders

In [3]:
import os
download_data = False
if not os.path.exists(config.DATA_SAVE_DIR):
    os.makedirs(config.DATA_SAVE_DIR)
    download_data = True
if not os.path.exists(config.DATA_SAVE_DIR_SINGLETICKER):
    os.makedirs(config.DATA_SAVE_DIR_SINGLETICKER)
    download_data = True
if not os.path.exists(config.TRAINED_MODEL_DIR):
    os.makedirs("./" + config.TRAINED_MODEL_DIR)
if not os.path.exists(config.TENSORBOARD_LOG_DIR):
    os.makedirs(config.TENSORBOARD_LOG_DIR)
if not os.path.exists(config.RESULTS_DIR):
    os.makedirs(config.RESULTS_DIR)

<a id='2'></a>
# Part 3. Download Data

In [4]:
data_downloader_singleticker = CryptoDownloader_binance(config.START_DATE, config.END_DATE, config.SINGLE_TICKER, config.DATA_SAVE_DIR_SINGLETICKER, config.DATA_GRANULARITY)
data_downloader_multipleticker = CryptoDownloader_binance(config.START_DATE, config.END_DATE, config.MULTIPLE_TICKER_8, config.DATA_SAVE_DIR, config.DATA_GRANULARITY)
if download_data:    
    data_downloader_singleticker.download_data()
    data_downloader_multipleticker.download_data()
df_singleticker = data_downloader_singleticker.load()
df_multipleticker = data_downloader_multipleticker.load()

# Part 4: Preprocess Data

In [5]:
fe = FeatureEngineer(
                    use_technical_indicator=True,
                    use_turbulence=False,
                    user_defined_feature = False)

df_singleticker = fe.preprocess_data(df_singleticker)
df_multipleticker = fe.preprocess_data(df_multipleticker)

Successfully added technical indicators
Successfully added technical indicators


In [7]:
df_singleticker

,date,open,high,low,close,volume,tic,macd,boll_ub,boll_lb,...,rsi_720,cci_24,cci_168,cci_720,dx_24,dx_168,dx_720,close_24_sma,close_168_sma,close_720_sma
0,2020-01-01 00:00:00,7195.24,7196.25,7175.46,7177.02,511.814901,btc,0.000000,7252.152882,7141.137118,...,100.000000,66.666667,66.666667,66.666667,100.000000,100.000000,100.000000,7177.020000,7177.020000,7177.020000
1,2020-01-01 01:00:00,7176.47,7230.00,7175.71,7216.27,883.052603,btc,0.880609,7252.152882,7141.137118,...,100.000000,66.666667,66.666667,66.666667,100.000000,100.000000,100.000000,7196.645000,7196.645000,7196.645000
2,2020-01-01 02:00:00,7215.52,7244.87,7211.41,7242.85,655.156809,btc,1.946315,7278.281843,7145.811491,...,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,7212.046667,7212.046667,7212.046667
3,2020-01-01 03:00:00,7242.66,7245.00,7220.00,7225.01,783.724867,btc,1.739213,7270.900272,7159.674728,...,78.640903,61.099666,61.099666,61.099666,100.000000,100.000000,100.000000,7215.287500,7215.287500,7215.287500
4,2020-01-01 04:00:00,7225.00,7230.00,7215.03,7217.27,467.812578,btc,1.282070,7263.878704,7167.489296,...,71.962976,25.208069,25.208069,25.208069,77.386626,80.952195,81.370828,7215.684000,7215.684000,7215.684000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10939,2021-03-31 19:00:00,59304.45,59567.28,58453.30,58553.12,3276.305903,btc,138.647494,59811.121507,57570.590493,...,51.869734,34.956619,106.887771,89.476052,0.195958,3.785988,4.757866,58684.857083,55643.739107,54802.982222
10940,2021-03-31 20:00:00,58553.12,59087.00,58328.55,58934.60,3283.190916,btc,141.272269,59805.344764,57570.675236,...,51.948387,21.037704,103.085298,87.688763,3.253055,3.210726,4.872046,58697.673750,55672.761429,54817.457500
10941,2021-03-31 21:00:00,58935.10,59289.45,58700.00,58940.50,1671.199727,btc,142.189431,59810.387163,57571.106837,...,51.949603,50.965802,108.059751,91.793347,2.039917,4.083830,4.667845,58713.383333,55703.657321,54831.563625
10942,2021-03-31 22:00:00,58940.50,59009.85,58838.86,58905.11,154.871347,btc,138.464480,59827.147769,57580.245231,...,51.941707,38.900282,104.967470,90.337844,2.039917,4.083830,4.667845,58722.662500,55741.580417,54845.129431


In [65]:
df_multipleticker

,date,open,high,low,close,volume,tic,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma
0,2020-01-01 00:00:00,0.032850,0.032850,0.032700,0.032780,1.166001e+06,ada,0.000000,0.033182,0.032588,100.000000,66.666667,100.000000,0.032780,0.032780
10944,2020-01-01 00:00:00,13.715900,13.721100,13.690300,13.698100,6.201669e+04,bnb,0.000000,0.033182,0.032588,100.000000,66.666667,100.000000,13.698100,13.698100
21888,2020-01-01 00:00:00,7195.240000,7196.250000,7175.460000,7177.020000,5.118149e+02,btc,0.000000,0.033182,0.032588,100.000000,66.666667,100.000000,7177.020000,7177.020000
32832,2020-01-01 00:00:00,0.002014,0.002023,0.002008,0.002008,9.630910e+05,doge,0.000000,0.033182,0.032588,100.000000,66.666667,100.000000,0.002008,0.002008
43776,2020-01-01 00:00:00,129.160000,129.190000,128.680000,128.870000,7.769173e+03,eth,0.000000,0.033182,0.032588,100.000000,66.666667,100.000000,128.870000,128.870000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43775,2021-03-31 23:00:00,0.053143,0.053911,0.053000,0.053770,4.448111e+07,doge,-0.000103,0.054076,0.052762,50.006619,-10.311768,28.977404,0.053646,0.053938
54719,2021-03-31 23:00:00,1903.970000,1924.210000,1901.620000,1919.370000,2.122478e+04,eth,26.214167,1948.647647,1762.109353,64.618395,164.610885,23.659415,1851.563000,1833.205500
65663,2021-03-31 23:00:00,28.571700,29.440100,28.550100,29.416700,4.557268e+05,link,0.276832,29.035767,26.359743,64.396467,226.662894,36.167462,27.746403,27.922328
76607,2021-03-31 23:00:00,194.620000,197.630000,194.470000,196.700000,2.863109e+04,ltc,0.332601,198.176985,188.501015,55.327678,76.382622,7.269085,194.139000,194.302667


## Add covariance matrix as states

In [163]:
# add covariance matrix as states
df_singleticker = df_singleticker.sort_values(['date','tic'],ignore_index=True)
df_singleticker.index = df_singleticker.date.factorize()[0]

cov_list = []
# look back is one year
lookback=252
for i in range(lookback,len(df_singleticker.index.unique())):
  data_lookback = df_singleticker.loc[i-lookback:i,:]
  price_lookback=data_lookback.pivot_table(index = 'date',columns = 'tic', values = 'close')
  return_lookback = price_lookback.pct_change().dropna()
  covs = return_lookback.cov().values 
  cov_list.append(covs)
  
df_cov = pd.DataFrame({'date':df_singleticker.date.unique()[lookback:],'cov_list':cov_list})
df_singleticker = df_singleticker.merge(df_cov, on='date')
df_singleticker = df_singleticker.sort_values(['date','tic']).reset_index(drop=True)
        

In [164]:
# add covariance matrix as states
df_multipleticker = df_multipleticker.sort_values(['date','tic'],ignore_index=True)
df_multipleticker.index = df_multipleticker.date.factorize()[0]

cov_list = []
# look back is one year
lookback=252
for i in range(lookback,len(df_multipleticker.index.unique())):
  data_lookback = df_multipleticker.loc[i-lookback:i,:]
  price_lookback=data_lookback.pivot_table(index = 'date',columns = 'tic', values = 'close')
  return_lookback = price_lookback.pct_change().dropna()
  covs = return_lookback.cov().values 
  cov_list.append(covs)
  
df_cov = pd.DataFrame({'date':df_multipleticker.date.unique()[lookback:],'cov_list':cov_list})
df_multipleticker = df_multipleticker.merge(df_cov, on='date')
df_multipleticker = df_multipleticker.sort_values(['date','tic']).reset_index(drop=True)
        

In [12]:
df_singleticker

,date,open,high,low,close,volume,tic,macd,boll_ub,boll_lb,...,cci_24,cci_168,cci_720,dx_24,dx_168,dx_720,close_24_sma,close_168_sma,close_720_sma,cov_list
0,2020-01-11 12:00:00,8045.89,8068.00,8018.41,8061.95,1788.465240,btc,33.263463,8225.486259,7967.135741,...,-26.004116,62.666768,86.279057,17.183610,18.122409,17.680470,8074.771667,7812.292857,7608.889921,[[4.219114843410537e-05]]
1,2020-01-11 13:00:00,8061.93,8102.56,8045.01,8090.02,1952.326847,btc,31.635774,8216.404682,7988.589318,...,-0.109997,69.824210,91.523086,24.959971,19.215235,18.218640,8083.922500,7816.972262,7610.784134,[[4.2127865541872805e-05]]
2,2020-01-11 14:00:00,8090.73,8135.00,8050.00,8053.75,2019.034562,btc,27.106664,8214.103674,7994.049326,...,-8.282975,69.287997,91.049828,31.520515,20.226632,18.718813,8087.051667,7821.359345,7612.521255,[[4.218371621024702e-05]]
3,2020-01-11 15:00:00,8054.00,8106.65,8053.08,8099.73,1461.322326,btc,26.917228,8214.136611,7998.040389,...,-1.461274,70.491345,91.837244,31.520515,20.226632,18.718813,8088.432083,7826.006607,7614.424414,[[4.2259558733250784e-05]]
4,2020-01-11 16:00:00,8099.16,8180.70,8070.00,8169.46,3939.308813,btc,32.024568,8221.329803,8000.477197,...,72.294389,84.199509,101.549776,44.586230,22.494963,19.844087,8097.465833,7831.139643,7616.584086,[[4.2512176878065345e-05]]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10687,2021-03-31 19:00:00,59304.45,59567.28,58453.30,58553.12,3276.305903,btc,138.647494,59811.121507,57570.590493,...,34.956619,106.887771,89.476052,0.195958,3.785988,4.757866,58684.857083,55643.739107,54802.982222,[[6.118955464622852e-05]]
10688,2021-03-31 20:00:00,58553.12,59087.00,58328.55,58934.60,3283.190916,btc,141.272269,59805.344764,57570.675236,...,21.037704,103.085298,87.688763,3.253055,3.210726,4.872046,58697.673750,55672.761429,54817.457500,[[6.131124618368812e-05]]
10689,2021-03-31 21:00:00,58935.10,59289.45,58700.00,58940.50,1671.199727,btc,142.189431,59810.387163,57571.106837,...,50.965802,108.059751,91.793347,2.039917,4.083830,4.667845,58713.383333,55703.657321,54831.563625,[[5.952863404059246e-05]]
10690,2021-03-31 22:00:00,58940.50,59009.85,58838.86,58905.11,154.871347,btc,138.464480,59827.147769,57580.245231,...,38.900282,104.967470,90.337844,2.039917,4.083830,4.667845,58722.662500,55741.580417,54845.129431,[[5.949951153658715e-05]]


In [165]:
df_multipleticker

,date,open,high,low,close,volume,tic,macd,boll_ub,boll_lb,...,cci_168,cci_720,dx_24,dx_168,dx_720,close_24_sma,close_168_sma,close_720_sma,cov_list_x,cov_list_y
0,2020-01-22 00:00:00,0.046040,0.046440,0.045750,0.046190,1.555857e+07,ada,0.000404,0.046782,0.044750,...,123.561621,152.547164,21.061190,19.439364,18.465198,0.045631,0.043302,0.038321,"[[0.00011398669795092598, 7.12229111080122e-05...","[[0.00011398669795092598, 7.12229111080122e-05..."
1,2020-01-22 00:00:00,18.110700,18.277900,18.062700,18.260600,1.232057e+05,bnb,0.183479,18.226752,17.206848,...,120.431374,131.906913,36.191356,6.666408,9.488483,17.678900,17.488811,15.489400,"[[0.00011398669795092598, 7.12229111080122e-05...","[[0.00011398669795092598, 7.12229111080122e-05..."
2,2020-01-22 00:00:00,8736.040000,8758.080000,8722.360000,8739.000000,1.020101e+03,btc,13.836927,8760.624515,8567.770485,...,-16.427892,83.074826,3.054436,5.717458,9.819945,8664.613750,8769.329881,8102.697683,"[[0.00011398669795092598, 7.12229111080122e-05...","[[0.00011398669795092598, 7.12229111080122e-05..."
3,2020-01-22 00:00:00,0.002358,0.002376,0.002355,0.002364,7.710088e+06,doge,-0.000007,0.002396,0.002345,...,-46.774887,77.594727,35.434854,3.488257,9.892601,0.002372,0.002403,0.002213,"[[0.00011398669795092598, 7.12229111080122e-05...","[[0.00011398669795092598, 7.12229111080122e-05..."
4,2020-01-22 00:00:00,169.480000,170.050000,169.300000,169.710000,9.174596e+03,eth,0.547117,170.135071,165.624929,...,46.531824,106.730994,12.368675,5.373139,13.013863,167.840833,167.442440,148.745703,"[[0.00011398669795092598, 7.12229111080122e-05...","[[0.00011398669795092598, 7.12229111080122e-05..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83515,2021-03-31 23:00:00,0.053143,0.053911,0.053000,0.053770,4.448111e+07,doge,-0.000103,0.054076,0.052762,...,9.262864,-35.614413,29.899575,13.848862,2.668755,0.053532,0.053454,0.054938,"[[0.0001184825404164931, 7.097453732794227e-05...","[[0.0001184825404164931, 7.097453732794227e-05..."
83516,2021-03-31 23:00:00,1903.970000,1924.210000,1901.620000,1919.370000,2.122478e+04,eth,26.214167,1948.647647,1762.109353,...,162.510039,134.557923,24.739908,11.755890,5.306469,1853.530000,1724.876548,1730.304278,"[[0.0001184825404164931, 7.097453732794227e-05...","[[0.0001184825404164931, 7.097453732794227e-05..."
83517,2021-03-31 23:00:00,28.571700,29.440100,28.550100,29.416700,4.557268e+05,link,0.276832,29.035767,26.359743,...,185.849024,33.412581,43.938434,3.029319,9.298379,27.699504,26.996359,28.499531,"[[0.0001184825404164931, 7.097453732794227e-05...","[[0.0001184825404164931, 7.097453732794227e-05..."
83518,2021-03-31 23:00:00,194.620000,197.630000,194.470000,196.700000,2.863109e+04,ltc,0.332601,198.176985,188.501015,...,101.102391,16.698944,8.903654,2.532939,6.693301,193.808750,185.852024,193.811569,"[[0.0001184825404164931, 7.097453732794227e-05...","[[0.0001184825404164931, 7.097453732794227e-05..."


<a id='4'></a>
# Part 5. Design Environment
Considering the stochastic and interactive nature of the automated stock trading tasks, a financial task is modeled as a **Markov Decision Process (MDP)** problem. The training process involves observing stock price change, taking an action and reward's calculation to have the agent adjusting its strategy accordingly. By interacting with the environment, the trading agent will derive a trading strategy with the maximized rewards as time proceeds.

Our trading environments, based on OpenAI Gym framework, simulate live stock markets with real market data according to the principle of time-driven simulation.

The action space describes the allowed actions that the agent interacts with the environment. Normally, action a includes three actions: {-1, 0, 1}, where -1, 0, 1 represent selling, holding, and buying one share. Also, an action can be carried upon multiple shares. We use an action space {-k,…,-1, 0, 1, …, k}, where k denotes the number of shares to buy and -k denotes the number of shares to sell. For example, "Buy 10 shares of AAPL" or "Sell 10 shares of AAPL" are 10 or -10, respectively. The continuous action space needs to be normalized to [-1, 1], since the policy is defined on a Gaussian distribution, which needs to be normalized and symmetric.

## Training data split: 2020-01-01 to 2020-12-31

In [203]:
train_singleticker = data_split(df_singleticker, '2020-01-01','2020-12-31')

In [166]:
train_multipleticker = data_split(df_multipleticker, '2020-01-01','2020-12-31')

## Environment for Portfolio Allocation


In [261]:
stock_dimension = len(train_singleticker.tic.unique())
state_space = 1 + 2*stock_dimension + len(config.TECHNICAL_INDICATORS_LIST)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

env_kwargs_singleticker = {
    "initial_amount": 1000000, 
    "technical_indicator_list": config.TECHNICAL_INDICATORS_LIST, 
    "max_assets_amount_per_trade": 100, 
    "main_tickers": config.SINGLE_TICKER,
    "all_tickers": config.SINGLE_TICKER,
    "reward_scaling": 1e-4,
    "comission_value": 0.1,
    "if_discrete":True
    
}

e_train_gym_singleticker = CustomTradingEnv(df = train_singleticker, **env_kwargs_singleticker)

env_train_discreteactionspace, _ = e_train_gym_singleticker.get_sb_env()

Stock Dimension: 1, State Space: 18
[0.0]


In [262]:
stock_dimension = len(train_multipleticker.tic.unique())
state_space = stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

env_kwargs_multipleticker = {
    "initial_amount": 1000000, 
    "technical_indicator_list": config.TECHNICAL_INDICATORS_LIST, 
    "max_assets_amount_per_trade": 100, 
    "main_tickers": config.MULTIPLE_TICKER_8,
    "all_tickers": config.MULTIPLE_TICKER_8,
    "reward_scaling": 1e-4,
    "comission_value": 0.1,
    "if_discrete":False
    
}

e_train_gym_multipleticker = CustomTradingEnv(df = train_multipleticker, **env_kwargs_multipleticker)

env_train_continousactionspace, _ = e_train_gym_multipleticker.get_sb_env()

Stock Dimension: 8, State Space: 8
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


<a id='5'></a>
# Part 6: Implement DRL Algorithms

In [263]:
# initialize
agent_singleticker = DRLAgent(env = env_train_discreteactionspace)

In [264]:
# initialize
agent_multipleticker = DRLAgent(env = env_train_continousactionspace)

### Model 1: **DQN**

In [265]:
model_dqn = agent_singleticker.get_model(model_name="dqn",model_kwargs=config.DQN_PARAMS)

{'batch_size': 128, 'buffer_size': 50000, 'learning_rate': 0.001}
Using cuda device


In [266]:
trained_dqn = agent_singleticker.train_model(model=model_dqn, 
                                tb_log_name='dqn',
                                total_timesteps=100)

[0.0]
Logging to ./tensorboard_log/dqn\dqn_19
[0.0]
[100.0]
[100.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]
[104.0]


### Model 2: **A2C**


In [267]:
A2C_PARAMS = {"n_steps": 5, "ent_coef": 0.005, "learning_rate": 0.0002}
model_a2c = agent_multipleticker.get_model(model_name="a2c",model_kwargs = A2C_PARAMS)

{'n_steps': 5, 'ent_coef': 0.005, 'learning_rate': 0.0002}
Using cuda device


In [268]:
trained_a2c = agent_multipleticker.train_model(model=model_a2c, 
                                tb_log_name='a2c',
                                total_timesteps=100)

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Logging to ./tensorboard_log/a2c\a2c_13
[0.0, 0.0, 100.0, 0.0, 32.541224360466, 0.0, 100.0, 0.0]
[0.0, 0.0, 20.547902584075928, 0.0, 36.70179173350334, 100.0, 31.511014699935913, 43.231695890426636]
[89.3456220626831, 0.0, 120.54790258407593, 12.560147047042847, 18.69257465004921, 33.157169818878174, 0.0, 32.12079107761383]
[49.784448742866516, 0.0, 20.547902584075928, 0.0, 11.657998710870743, 79.59666848182678, 0.0, 132.12079107761383]
[18.346264958381653, 100.0, 120.54790258407593, 0.0, 0.0, 69.28643584251404, 0.0, 61.486563086509705]
[0.39018839597702026, 200.0, 20.547902584075928, 100.0, 96.41000032424927, 98.76700937747955, 100.0, 0.0]
[0.0, 300.0, 0.0, 200.0, 12.490767240524292, 120.84024846553802, 148.66427779197693, 0.0]
[0.0, 219.99040246009827, 0.0, 262.9776418209076, 0.0, 205.91919720172882, 248.66427779197693, 0.0]
[0.0, 268.3763265609741, 0.0, 162.9776418209076, 0.0, 299.6351271867752, 209.71416234970093, 100.0]
[0.0, 285.4188591241

### Model 3: **PPO**

In [43]:
# agent = DRLAgent(env = env_train)
# PPO_PARAMS = {
#     "n_steps": 2048,
#     "ent_coef": 0.005,
#     "learning_rate": 0.0001,
#     "batch_size": 128,
# }
# model_ppo = agent.get_model("ppo",model_kwargs = PPO_PARAMS)

In [44]:
# trained_ppo = agent.train_model(model=model_ppo, 
#                              tb_log_name='ppo',
#                              total_timesteps=80000)

## Trading
Assume that we have $1,000,000 initial capital at 2021-01-01.

In [269]:
trade_dqn = data_split(df_singleticker,config.START_TEST_DATE, config.END_DATE)
e_trade_gym_dqn = CustomTradingEnv(df = trade_dqn, **env_kwargs_singleticker)

trade_a2c = data_split(df_multipleticker,config.START_TEST_DATE, config.END_DATE)
e_trade_gym_a2c = CustomTradingEnv(df = trade_a2c, **env_kwargs_multipleticker)

In [270]:
trade_dqn.shape

(2160, 24)

In [271]:
trade_a2c.shape

(17280, 24)

In [272]:
trade_dqn

,date,open,high,low,close,volume,tic,macd,boll_ub,boll_lb,...,cci_168,cci_720,dx_24,dx_168,dx_720,close_24_sma,close_168_sma,close_720_sma,cov_list_x,cov_list_y
0,2021-01-01 00:00:00,28923.63,29031.34,28690.17,28995.13,2311.811445,btc,133.938215,29333.932277,28372.364723,...,118.691744,158.444837,7.810386,2.973777,4.289133,28857.474167,26771.040298,21913.602653,[[6.768847713583873e-05]],[[6.768847713583873e-05]]
1,2021-01-01 01:00:00,28995.13,29470.00,28960.35,29409.99,5403.068471,btc,165.296812,29414.880693,28335.195307,...,138.392660,166.245865,15.340298,6.115833,5.246867,28885.241667,26805.640179,21928.263736,[[6.759145166361914e-05]],[[6.759145166361914e-05]]
2,2021-01-01 02:00:00,29410.00,29465.26,29120.03,29194.65,2384.231560,btc,170.803670,29450.117290,28333.522710,...,136.321103,165.133874,15.340298,6.115833,5.246867,28904.998333,26838.958274,21942.872417,[[6.786669199582877e-05]],[[6.786669199582877e-05]]
3,2021-01-01 03:00:00,29195.25,29367.00,29150.02,29278.40,1461.345077,btc,179.852591,29471.029494,28324.925506,...,135.731228,164.603091,15.340298,6.115833,5.246867,28915.598333,26873.465357,21957.678181,[[6.787552711412387e-05]],[[6.787552711412387e-05]]
4,2021-01-01 04:00:00,29278.41,29395.00,29029.40,29220.31,2038.046803,btc,180.258639,29503.594628,28321.760372,...,131.940416,162.845715,7.210698,5.145507,4.955662,28923.596667,26906.843512,21972.496722,[[6.790780319669332e-05]],[[6.790780319669332e-05]]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2155,2021-03-31 19:00:00,59304.45,59567.28,58453.30,58553.12,3276.305903,btc,138.647494,59811.121507,57570.590493,...,106.887771,89.476052,0.195958,3.785988,4.757866,58684.857083,55643.739107,54802.982222,[[6.118955464622852e-05]],[[6.118955464622852e-05]]
2156,2021-03-31 20:00:00,58553.12,59087.00,58328.55,58934.60,3283.190916,btc,141.272269,59805.344764,57570.675236,...,103.085298,87.688763,3.253055,3.210726,4.872046,58697.673750,55672.761429,54817.457500,[[6.131124618368812e-05]],[[6.131124618368812e-05]]
2157,2021-03-31 21:00:00,58935.10,59289.45,58700.00,58940.50,1671.199727,btc,142.189431,59810.387163,57571.106837,...,108.059751,91.793347,2.039917,4.083830,4.667845,58713.383333,55703.657321,54831.563625,[[5.952863404059246e-05]],[[5.952863404059246e-05]]
2158,2021-03-31 22:00:00,58940.50,59009.85,58838.86,58905.11,154.871347,btc,138.464480,59827.147769,57580.245231,...,104.967470,90.337844,2.039917,4.083830,4.667845,58722.662500,55741.580417,54845.129431,[[5.949951153658715e-05]],[[5.949951153658715e-05]]


In [273]:
trade_a2c

,date,open,high,low,close,volume,tic,macd,boll_ub,boll_lb,...,cci_168,cci_720,dx_24,dx_168,dx_720,close_24_sma,close_168_sma,close_720_sma,cov_list_x,cov_list_y
0,2021-01-01 00:00:00,0.181340,0.181460,0.178310,0.180510,1.919492e+07,ada,-0.000153,0.184657,0.176516,...,60.766462,171.281882,8.915525,5.439885,0.247604,0.180321,0.169346,0.157786,"[[0.00023491493831790994, 7.732551281244585e-0...","[[0.00023491493831790994, 7.732551281244585e-0..."
0,2021-01-01 00:00:00,37.359600,37.442300,36.963600,37.376400,9.511383e+04,bnb,-0.083237,37.946399,36.719461,...,63.526428,157.989939,12.942067,8.590561,3.405324,37.327438,35.431929,31.413674,"[[0.00023491493831790994, 7.732551281244585e-0...","[[0.00023491493831790994, 7.732551281244585e-0..."
0,2021-01-01 00:00:00,28923.630000,29031.340000,28690.170000,28995.130000,2.311811e+03,btc,133.938215,29333.932277,28372.364723,...,118.691744,158.444837,7.810386,2.973777,4.289133,28857.474167,26771.040298,21913.602653,"[[0.00023491493831790994, 7.732551281244585e-0...","[[0.00023491493831790994, 7.732551281244585e-0..."
0,2021-01-01 00:00:00,0.004672,0.004701,0.004601,0.004679,2.768207e+07,doge,0.000018,0.004729,0.004558,...,102.515356,101.529522,4.780309,5.513748,9.894779,0.004637,0.004560,0.003808,"[[0.00023491493831790994, 7.732551281244585e-0...","[[0.00023491493831790994, 7.732551281244585e-0..."
0,2021-01-01 00:00:00,736.420000,739.000000,729.330000,734.070000,2.793270e+04,eth,-0.496311,752.068407,727.317593,...,63.143871,179.597339,20.122001,3.900295,0.104865,739.810833,691.517857,620.489833,"[[0.00023491493831790994, 7.732551281244585e-0...","[[0.00023491493831790994, 7.732551281244585e-0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2159,2021-03-31 23:00:00,0.053143,0.053911,0.053000,0.053770,4.448111e+07,doge,-0.000103,0.054076,0.052762,...,9.262864,-35.614413,29.899575,13.848862,2.668755,0.053532,0.053454,0.054938,"[[0.0001184825404164931, 7.097453732794227e-05...","[[0.0001184825404164931, 7.097453732794227e-05..."
2159,2021-03-31 23:00:00,1903.970000,1924.210000,1901.620000,1919.370000,2.122478e+04,eth,26.214167,1948.647647,1762.109353,...,162.510039,134.557923,24.739908,11.755890,5.306469,1853.530000,1724.876548,1730.304278,"[[0.0001184825404164931, 7.097453732794227e-05...","[[0.0001184825404164931, 7.097453732794227e-05..."
2159,2021-03-31 23:00:00,28.571700,29.440100,28.550100,29.416700,4.557268e+05,link,0.276832,29.035767,26.359743,...,185.849024,33.412581,43.938434,3.029319,9.298379,27.699504,26.996359,28.499531,"[[0.0001184825404164931, 7.097453732794227e-05...","[[0.0001184825404164931, 7.097453732794227e-05..."
2159,2021-03-31 23:00:00,194.620000,197.630000,194.470000,196.700000,2.863109e+04,ltc,0.332601,198.176985,188.501015,...,101.102391,16.698944,8.903654,2.532939,6.693301,193.808750,185.852024,193.811569,"[[0.0001184825404164931, 7.097453732794227e-05...","[[0.0001184825404164931, 7.097453732794227e-05..."


In [274]:
allocations_dqn, transactions_dqn, allocation_values_dqn = agent_singleticker.DRL_prediction(model=trained_dqn,
                        environment = e_trade_gym_dqn)

[0.0]
[0.0]
[0.0]
[0.0]
[0.0]
[0.0]
[0.0]
[0.0]
[0.0]
[0.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[31.0]
[3

In [275]:
allocations_dqn

,cash,btc,date
0,1000000.000,0.0,2021-01-01 01:00:00
0,1000000.000,0.0,2021-01-01 02:00:00
0,1000000.000,0.0,2021-01-01 03:00:00
0,1000000.000,0.0,2021-01-01 04:00:00
0,1000000.000,0.0,2021-01-01 05:00:00
...,...,...,...
0,11099.659,31.0,2021-03-31 18:00:00
0,11099.659,31.0,2021-03-31 19:00:00
0,11099.659,31.0,2021-03-31 20:00:00
0,11099.659,31.0,2021-03-31 21:00:00


In [276]:
transactions_dqn

,date,operation,tic,amount,price
0,2021-01-01 08:00:00,buy,btc,31.0,29000.01


In [277]:
allocation_values_dqn

,cash,btc,date
0,1000000.000,0.00,2021-01-01 01:00:00
0,1000000.000,0.00,2021-01-01 02:00:00
0,1000000.000,0.00,2021-01-01 03:00:00
0,1000000.000,0.00,2021-01-01 04:00:00
0,1000000.000,0.00,2021-01-01 05:00:00
...,...,...,...
0,11099.659,1838282.95,2021-03-31 18:00:00
0,11099.659,1815146.72,2021-03-31 19:00:00
0,11099.659,1826972.60,2021-03-31 20:00:00
0,11099.659,1827155.50,2021-03-31 21:00:00


In [278]:
allocations_a2c, transactions_a2c, allocation_values_a2c = agent_multipleticker.DRL_prediction(model=trained_a2c,
                        environment = e_trade_gym_a2c)

.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.

In [280]:
allocations_a2c

,cash,btc,eth,bnb,ada,xrp,doge,link,ltc,date
0,68.499676,31.0,13.000000,0.000000,94.802988,0.000000,49.963832,0.000000,5.0,2021-01-01 01:00:00
0,827265.104099,0.0,0.000000,65.257668,0.000000,22.339532,0.000000,0.000000,0.0,2021-01-01 02:00:00
0,828773.630837,0.0,0.000000,10.275936,100.000000,122.339532,0.000000,25.525442,0.0,2021-01-01 03:00:00
0,745147.756841,0.0,100.000000,54.913202,200.000000,22.339532,22.598413,9.461634,0.0,2021-01-01 04:00:00
0,79185.490472,22.0,32.238871,154.913202,300.000000,65.240040,0.000000,0.000000,0.0,2021-01-01 05:00:00
...,...,...,...,...,...,...,...,...,...,...
0,0.007409,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,2021-03-31 18:00:00
0,0.007409,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,2021-03-31 19:00:00
0,0.007409,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,2021-03-31 20:00:00
0,0.007409,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,2021-03-31 21:00:00


In [281]:
transactions_a2c

,date,operation,tic,amount,price
0,2021-01-01 00:00:00,buy,btc,31.000000,28995.130000
0,2021-01-01 00:00:00,buy,eth,13.000000,734.070000
0,2021-01-01 00:00:00,buy,ada,94.802988,0.180510
0,2021-01-01 00:00:00,buy,doge,49.963832,0.004679
0,2021-01-01 00:00:00,buy,ltc,5.000000,123.870000
...,...,...,...,...,...
0,2021-01-17 08:00:00,sell,doge,1.000000,0.008937
0,2021-01-21 14:00:00,buy,doge,1.000000,0.008296
0,2021-01-21 18:00:00,sell,doge,1.000000,0.008438
0,2021-01-28 02:00:00,buy,doge,0.257839,0.007679


In [282]:
allocation_values_a2c

,cash,btc,eth,bnb,ada,xrp,doge,link,ltc,date
0,68.499676,911709.69,9727.640000,0.000000,17.403932,0.000000,0.236654,0.000000,658.5,2021-01-01 01:00:00
0,827265.104099,0.00,0.000000,2477.181096,0.000000,4.995343,0.000000,0.000000,0.0,2021-01-01 02:00:00
0,828773.630837,0.00,0.000000,389.714878,18.292000,27.807776,0.000000,293.557896,0.0,2021-01-01 03:00:00
0,745147.756841,0.00,74229.000000,2079.573940,36.360000,5.338701,0.110592,108.406670,0.0,2021-01-01 04:00:00
0,79185.490472,642114.22,23877.719702,5842.226093,54.891000,15.539525,0.000000,0.000000,0.0,2021-01-01 05:00:00
...,...,...,...,...,...,...,...,...,...,...
0,0.007409,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,2021-03-31 18:00:00
0,0.007409,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,2021-03-31 19:00:00
0,0.007409,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,2021-03-31 20:00:00
0,0.007409,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,2021-03-31 21:00:00


<a id='6'></a>
# Part 7: Backtest

In [288]:
%load_ext autoreload
%autoreload 
bat = BackTest(trained_dqn, e_trade_gym_dqn)
bat.create_summary(allocations_dqn, allocation_values_dqn, transactions_dqn)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Start date,2021-01-01
End date,2021-03-31
Total months,4
,Backtest
Annual return,592.627%
Cumulative returns,99.608%
Annual volatility,77.302%
Sharpe ratio,2.92
Calmar ratio,23.75
Stability,0.78
Max drawdown,-24.955%


Worst drawdown periods,Net drawdown in %,Peak date,Valley date,Recovery date,Duration
0,24.95,2021-01-08,2021-01-27,2021-02-08,22
1,21.25,2021-02-21,2021-02-28,2021-03-11,14
2,16.08,2021-03-13,2021-03-25,NaT,NaN
3,3.45,2021-02-09,2021-02-10,2021-02-11,3
4,3.03,2021-01-03,2021-01-04,2021-01-05,2


Stress Events,mean,min,max
New Normal,0.89%,-12.89%,19.36%


Top 10 long positions of all time,max
btc,0.28%


Top 10 short positions of all time,max


Top 10 positions of all time,max
btc,0.28%
index,0.00%


In [289]:
%load_ext autoreload
%autoreload 
bat = BackTest(trained_a2c, e_trade_gym_a2c)
bat.create_summary(allocations_a2c, allocation_values_a2c, transactions_a2c)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Start date,2021-01-01
End date,2021-03-31
Total months,4
,Backtest
Annual return,-100.0%
Cumulative returns,-100.0%
Annual volatility,513.276%
Sharpe ratio,-6.79
Calmar ratio,-1.00
Stability,NaN
Max drawdown,-100.0%


Worst drawdown periods,Net drawdown in %,Peak date,Valley date,Recovery date,Duration
0,100.00,2021-01-01,2021-01-16,NaT,NaN
1,NaN,NaT,NaT,NaT,NaN
2,NaN,NaT,NaT,NaT,NaN
3,NaN,NaT,NaT,NaT,NaN
4,NaN,NaT,NaT,NaT,NaN


Stress Events,mean,min,max
New Normal,-13.83%,-190.06%,0.00%


Top 10 long positions of all time,max
doge,99.99%
xrp,71.58%
ada,52.47%
ltc,12.70%
btc,11.82%
link,9.72%
eth,4.96%
bnb,3.47%


Top 10 short positions of all time,max


Top 10 positions of all time,max
doge,99.99%
xrp,71.58%
ada,52.47%
ltc,12.70%
btc,11.82%
link,9.72%
eth,4.96%
bnb,3.47%
index,0.00%


In [291]:
bat.plot_return_against_hold(allocation_values_dqn)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed